# Data Setup

In [1]:
import torch

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [10]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [12]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [13]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [14]:
x_train_tensor = torch.from_numpy(x_train)
x_test_tensor = torch.from_numpy(x_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

# Defining Model

In [25]:
class NeuralNetwork():

    def __init__(self, x: torch.Tensor):
        self.weights = torch.rand(x.shape[1], 1, dtype = torch.float64, requires_grad = True)
        self.bias = torch.zeros(1, 1, dtype = torch.float64, requires_grad = True)
    
    def forward(self, x: torch.Tensor):
        z = torch.matmul(x, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred
    
    def loss_function(self, y_pred: torch.Tensor, y: torch.Tensor):
        #Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        # Calculate loss
        loss = -(y* torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
        return loss

# Parameters

In [46]:
lr = 0.1
epochs = 1000

# Training Pipeline

In [47]:
model = NeuralNetwork(x_train_tensor)

# Define loop
for epoch in range(epochs):
    # Forward Pass
    y_pred = model.forward(x_train_tensor)

    # Loss Calculation
    loss = model.loss_function(y_pred, y_train_tensor)
    print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")

    # Backward Pass
    loss.backward()

    # Paramers Update
    with torch.no_grad():
        model.weights -= lr * model.weights.grad
        model.bias -= lr * model.bias.grad
    
    # Zero Gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()

Epoch: 1, Loss: 3.7250061701931445
Epoch: 2, Loss: 3.605434621710056
Epoch: 3, Loss: 3.481513389996679
Epoch: 4, Loss: 3.352395921611901
Epoch: 5, Loss: 3.219366017972275
Epoch: 6, Loss: 3.0828896762389375
Epoch: 7, Loss: 2.941938937028931
Epoch: 8, Loss: 2.7947137813291203
Epoch: 9, Loss: 2.648121709999587
Epoch: 10, Loss: 2.5013506724259997
Epoch: 11, Loss: 2.3548032314361103
Epoch: 12, Loss: 2.2064111544523453
Epoch: 13, Loss: 2.0622824592859352
Epoch: 14, Loss: 1.9232038985471407
Epoch: 15, Loss: 1.7860976902565031
Epoch: 16, Loss: 1.6565431140772702
Epoch: 17, Loss: 1.5290427790506023
Epoch: 18, Loss: 1.4112420534097343
Epoch: 19, Loss: 1.3068914507265557
Epoch: 20, Loss: 1.2167380432056665
Epoch: 21, Loss: 1.140838561502456
Epoch: 22, Loss: 1.0783158901197891
Epoch: 23, Loss: 1.0274655711533978
Epoch: 24, Loss: 0.986204270356751
Epoch: 25, Loss: 0.9525251854754976
Epoch: 26, Loss: 0.9247298543217285
Epoch: 27, Loss: 0.9014693118654178
Epoch: 28, Loss: 0.8817019590711965
Epoch: 29

# Evaluation

In [61]:
with torch.no_grad():
    y_pred = model.forward(x_test_tensor)
    y_pred = (y_pred > 0.7).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.6491228342056274
